In [1]:
import numpy as np
import pandas as pd
import tensorflow as tf

In [2]:
import tensorflow_datasets as tfds

ratings= tfds.load("movielens/100k-ratings", split = "train")
movies= tfds.load("movielens/100k-movies", split = "train")

Shuffling and writing examples to /root/tensorflow_datasets/movielens/100k-ratings/0.1.0.incompleteOSUHNY/movielens-train.tfrecord


Dataset movielens downloaded and prepared to /root/tensorflow_datasets/movielens/100k-ratings/0.1.0. Subsequent calls will reuse this data.


Shuffling and writing examples to /root/tensorflow_datasets/movielens/100k-movies/0.1.0.incomplete5355KK/movielens-train.tfrecord


Dataset movielens downloaded and prepared to /root/tensorflow_datasets/movielens/100k-movies/0.1.0. Subsequent calls will reuse this data.


In [3]:
import tensorflow_hub as hub

embed = hub.load("https://tfhub.dev/google/Wiki-words-250-with-normalization/2")
embeddings = embed(["cat is on the mat", "dog is in the fog"])

In [4]:
assert isinstance(ratings, tf.data.Dataset)

In [5]:
ratings_df = tfds.as_dataframe(ratings)

In [6]:
ratings_df.head()

,bucketized_user_age,movie_genres,movie_id,movie_title,raw_user_age,timestamp,user_gender,user_id,user_occupation_label,user_occupation_text,user_rating,user_zip_code
0,45.0,[7],b'357',"b""One Flew Over the Cuckoo's Nest (1975)""",46.0,879024327,True,b'138',4,b'doctor',4.0,b'53211'
1,25.0,"[4, 14]",b'709',b'Strictly Ballroom (1992)',32.0,875654590,True,b'92',5,b'entertainment',2.0,b'80525'
2,18.0,[4],b'412',"b'Very Brady Sequel, A (1996)'",24.0,882075110,True,b'301',17,b'student',4.0,b'55439'
3,50.0,"[5, 7]",b'56',b'Pulp Fiction (1994)',50.0,883326919,True,b'60',4,b'healthcare',4.0,b'06472'
4,50.0,"[10, 16]",b'895',b'Scream 2 (1997)',55.0,891409199,True,b'197',18,b'technician',3.0,b'75094'


In [9]:
ratings_df = ratings_df[['user_id','movie_id', 'movie_title','user_rating']]
ratings_df.head()

# movie_title_vector_df['movie_title'] = movie_title_vector_df['movie_title'].apply(lambda x: x.decode("utf-8"))
# movie_title_vector_df.head()

,user_id,movie_id,movie_title,user_rating
0,138,357,One Flew Over the Cuckoo's Nest (1975),4.0
1,92,709,Strictly Ballroom (1992),2.0
2,301,412,"Very Brady Sequel, A (1996)",4.0
3,60,56,Pulp Fiction (1994),4.0
4,197,895,Scream 2 (1997),3.0


In [10]:
ratings_df['user_id'] = ratings_df['user_id'].apply(lambda x: x.decode('utf-8'))
ratings_df['movie_id'] = ratings_df['movie_id'].apply(lambda x: x.decode('utf-8'))
ratings_df['movie_title'] = ratings_df['movie_title'].apply(lambda x: x.decode('utf-8'))
ratings_df.head()

AttributeError: ignored

# 데이터 준비가 끝났으니깐 영화-사용자 평점 행렬만들기

In [11]:
movie_user_ratings_df = ratings_df[['user_id','movie_id','movie_title','user_rating']].pivot_table(index = 'movie_id', columns = 'user_id', values = 'user_rating')
movie_user_ratings_df

user_id,1,10,100,101,102,103,104,105,106,107,108,109,11,110,111,112,113,114,115,116,117,118,119,12,120,121,122,123,124,125,126,127,128,129,13,130,131,132,133,134,...,912,913,914,915,916,917,918,919,92,920,921,922,923,924,925,926,927,928,929,93,930,931,932,933,934,935,936,937,938,939,94,940,941,942,943,95,96,97,98,99
movie_id,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
1,5.0,4.0,NaN,3.0,3.0,NaN,NaN,NaN,4.0,NaN,4.0,4.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,4.0,NaN,NaN,NaN,4.0,4.0,NaN,NaN,3.0,4.0,NaN,NaN,4.0,NaN,3.0,5.0,4.0,NaN,NaN,5.0,...,NaN,2.0,NaN,NaN,4.0,3.0,3.0,4.0,4.0,NaN,3.0,5.0,3.0,5.0,NaN,NaN,5.0,NaN,3.0,5.0,3.0,NaN,4.0,3.0,2.0,3.0,4.0,NaN,4.0,NaN,4.0,NaN,5.0,NaN,NaN,5.0,5.0,4.0,NaN,4.0
10,3.0,NaN,NaN,NaN,NaN,NaN,2.0,NaN,NaN,NaN,5.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
100,5.0,5.0,NaN,NaN,NaN,NaN,4.0,NaN,3.0,NaN,4.0,4.0,4.0,NaN,NaN,NaN,4.0,5.0,5.0,NaN,NaN,5.0,5.0,NaN,NaN,4.0,NaN,4.0,NaN,NaN,NaN,NaN,NaN,NaN,5.0,3.0,5.0,4.0,NaN,NaN,...,NaN,3.0,NaN,NaN,5.0,4.0,NaN,5.0,5.0,NaN,NaN,NaN,5.0,4.0,NaN,NaN,NaN,NaN,4.0,NaN,3.0,4.0,5.0,5.0,4.0,3.0,4.0,3.0,5.0,NaN,5.0,3.0,NaN,NaN,5.0,NaN,5.0,2.0,NaN,5.0
1000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,3.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1001,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
995,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,4.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2.0,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,3.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,3.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
996,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,3.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
997,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,4.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [12]:
movie_user_ratings_df.fillna(0,inplace = True)
movie_user_ratings_df.head()

user_id,1,10,100,101,102,103,104,105,106,107,108,109,11,110,111,112,113,114,115,116,117,118,119,12,120,121,122,123,124,125,126,127,128,129,13,130,131,132,133,134,...,912,913,914,915,916,917,918,919,92,920,921,922,923,924,925,926,927,928,929,93,930,931,932,933,934,935,936,937,938,939,94,940,941,942,943,95,96,97,98,99
movie_id,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
1,5.0,4.0,0.0,3.0,3.0,0.0,0.0,0.0,4.0,0.0,4.0,4.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,4.0,0.0,0.0,0.0,4.0,4.0,0.0,0.0,3.0,4.0,0.0,0.0,4.0,0.0,3.0,5.0,4.0,0.0,0.0,5.0,...,0.0,2.0,0.0,0.0,4.0,3.0,3.0,4.0,4.0,0.0,3.0,5.0,3.0,5.0,0.0,0.0,5.0,0.0,3.0,5.0,3.0,0.0,4.0,3.0,2.0,3.0,4.0,0.0,4.0,0.0,4.0,0.0,5.0,0.0,0.0,5.0,5.0,4.0,0.0,4.0
10,3.0,0.0,0.0,0.0,0.0,0.0,2.0,0.0,0.0,0.0,5.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
100,5.0,5.0,0.0,0.0,0.0,0.0,4.0,0.0,3.0,0.0,4.0,4.0,4.0,0.0,0.0,0.0,4.0,5.0,5.0,0.0,0.0,5.0,5.0,0.0,0.0,4.0,0.0,4.0,0.0,0.0,0.0,0.0,0.0,0.0,5.0,3.0,5.0,4.0,0.0,0.0,...,0.0,3.0,0.0,0.0,5.0,4.0,0.0,5.0,5.0,0.0,0.0,0.0,5.0,4.0,0.0,0.0,0.0,0.0,4.0,0.0,3.0,4.0,5.0,5.0,4.0,3.0,4.0,3.0,5.0,0.0,5.0,3.0,0.0,0.0,5.0,0.0,5.0,2.0,0.0,5.0
1000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,3.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1001,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [13]:
# user_movie_ratings_df = ratings_df[['uesr_id','movie_title','user_rating']].pivot_table(index = 'user_id', columns = 'movie_title', values = 'user_rating')
# user_movie_ratings_df.fillna(0, inplace = True)
# user_movie_ratings_df.head()

user_movie_ratings_df = movie_user_ratings_df.T
user_movie_ratings_df

movie_id,1,10,100,1000,1001,1002,1003,1004,1005,1006,1007,1008,1009,101,1010,1011,1012,1013,1014,1015,1016,1017,1018,1019,102,1020,1021,1022,1023,1024,1025,1026,1027,1028,1029,103,1030,1031,1032,1033,...,963,964,965,966,967,968,969,97,970,971,972,973,974,975,976,977,978,979,98,980,981,982,983,984,985,986,987,988,989,99,990,991,992,993,994,995,996,997,998,999
user_id,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
1,5.0,3.0,5.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,3.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,4.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,3.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
10,4.0,0.0,5.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,4.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,5.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
100,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,3.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
101,3.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,3.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.0,0.0,0.0,0.0,2.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
102,3.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,4.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,3.0,0.0,0.0,0.0,0.0,0.0,2.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,4.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,2.0,0.0,0.0,0.0,2.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
95,5.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,3.0,0.0,4.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,5.0,0.0,4.0,0.0,3.0,0.0,0.0,0.0,0.0,2.0,0.0,0.0,0.0,4.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,4.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
96,5.0,0.0,5.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,5.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
97,4.0,0.0,2.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,5.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,4.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [14]:
m1 = movie_user_ratings_df.values > 0
m2 = movie_user_ratings_df.values <= 0
movie_user_watch_df = pd.DataFrame(np.select([m1,m2],[1.,0.]),index = movie_user_ratings_df.index, columns = movie_user_ratings_df.columns)
movie_user_watch_df.head()

user_id,1,10,100,101,102,103,104,105,106,107,108,109,11,110,111,112,113,114,115,116,117,118,119,12,120,121,122,123,124,125,126,127,128,129,13,130,131,132,133,134,...,912,913,914,915,916,917,918,919,92,920,921,922,923,924,925,926,927,928,929,93,930,931,932,933,934,935,936,937,938,939,94,940,941,942,943,95,96,97,98,99
movie_id,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
1,1.0,1.0,0.0,1.0,1.0,0.0,0.0,0.0,1.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,1.0,1.0,0.0,0.0,1.0,0.0,1.0,1.0,1.0,0.0,0.0,1.0,...,0.0,1.0,0.0,0.0,1.0,1.0,1.0,1.0,1.0,0.0,1.0,1.0,1.0,1.0,0.0,0.0,1.0,0.0,1.0,1.0,1.0,0.0,1.0,1.0,1.0,1.0,1.0,0.0,1.0,0.0,1.0,0.0,1.0,0.0,0.0,1.0,1.0,1.0,0.0,1.0
10,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
100,1.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,1.0,1.0,1.0,0.0,0.0,0.0,1.0,1.0,1.0,0.0,0.0,1.0,1.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,1.0,1.0,0.0,0.0,...,0.0,1.0,0.0,0.0,1.0,1.0,0.0,1.0,1.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,0.0,1.0,1.0,0.0,0.0,1.0,0.0,1.0,1.0,0.0,1.0
1000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1001,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


# 영화-사용자 평점 행렬을 사용해서 영화-영화 동시클릭 행렬 만들기

In [ ]:
# user_movie_watch_df[b"'Til There Was You (1997)"].sum()

#첫번째 영화를 본 사람의 수가 9명이다.

9.0

In [15]:
user_movie_watch_df = movie_user_watch_df.T

In [16]:
cowatch2 = np.dot(movie_user_watch_df, user_movie_watch_df)
cowatch_df2 = pd.DataFrame(cowatch2).head()
cowatch_df2.head()

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,...,1642,1643,1644,1645,1646,1647,1648,1649,1650,1651,1652,1653,1654,1655,1656,1657,1658,1659,1660,1661,1662,1663,1664,1665,1666,1667,1668,1669,1670,1671,1672,1673,1674,1675,1676,1677,1678,1679,1680,1681
0,452.0,61.0,325.0,8.0,14.0,6.0,7.0,4.0,10.0,12.0,25.0,26.0,43.0,67.0,30.0,72.0,83.0,27.0,76.0,10.0,105.0,33.0,20.0,20.0,45.0,20.0,20.0,13.0,20.0,2.0,17.0,2.0,1.0,116.0,10.0,14.0,16.0,6.0,13.0,30.0,...,30.0,5.0,14.0,22.0,10.0,14.0,58.0,179.0,5.0,18.0,23.0,2.0,26.0,36.0,10.0,36.0,21.0,27.0,244.0,17.0,7.0,12.0,11.0,23.0,17.0,15.0,2.0,47.0,17.0,134.0,13.0,11.0,3.0,58.0,4.0,12.0,8.0,12.0,14.0,7.0
1,61.0,89.0,74.0,1.0,3.0,2.0,2.0,0.0,9.0,7.0,14.0,12.0,22.0,15.0,15.0,21.0,13.0,3.0,16.0,4.0,15.0,12.0,10.0,8.0,7.0,13.0,16.0,4.0,3.0,3.0,6.0,1.0,1.0,19.0,0.0,5.0,4.0,0.0,1.0,5.0,...,9.0,4.0,6.0,7.0,2.0,3.0,12.0,38.0,5.0,16.0,10.0,0.0,4.0,6.0,2.0,10.0,8.0,9.0,48.0,7.0,4.0,4.0,6.0,10.0,10.0,2.0,1.0,9.0,4.0,25.0,7.0,4.0,1.0,4.0,1.0,6.0,1.0,2.0,5.0,0.0
2,325.0,74.0,508.0,7.0,16.0,6.0,5.0,6.0,17.0,20.0,37.0,36.0,53.0,55.0,43.0,86.0,86.0,31.0,76.0,11.0,103.0,50.0,25.0,24.0,32.0,29.0,29.0,16.0,22.0,4.0,17.0,1.0,1.0,113.0,7.0,10.0,12.0,4.0,10.0,23.0,...,36.0,6.0,17.0,22.0,10.0,13.0,49.0,185.0,5.0,24.0,23.0,3.0,24.0,34.0,10.0,40.0,24.0,31.0,290.0,20.0,6.0,15.0,12.0,24.0,20.0,18.0,2.0,46.0,15.0,125.0,14.0,14.0,3.0,39.0,5.0,14.0,6.0,11.0,12.0,5.0
3,8.0,1.0,7.0,10.0,4.0,2.0,1.0,0.0,0.0,1.0,1.0,0.0,0.0,4.0,1.0,4.0,2.0,2.0,4.0,0.0,6.0,2.0,1.0,1.0,2.0,0.0,0.0,0.0,3.0,0.0,1.0,0.0,1.0,6.0,1.0,0.0,4.0,1.0,3.0,1.0,...,1.0,1.0,0.0,1.0,0.0,0.0,3.0,8.0,0.0,1.0,0.0,0.0,2.0,2.0,2.0,3.0,1.0,1.0,5.0,0.0,0.0,2.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,7.0,1.0,0.0,2.0,3.0,0.0,0.0,5.0,3.0,3.0,4.0
4,14.0,3.0,16.0,4.0,17.0,4.0,1.0,0.0,0.0,0.0,2.0,1.0,2.0,1.0,2.0,6.0,6.0,2.0,4.0,1.0,5.0,6.0,1.0,2.0,0.0,0.0,0.0,2.0,3.0,0.0,4.0,1.0,1.0,11.0,0.0,3.0,2.0,0.0,4.0,6.0,...,1.0,1.0,0.0,0.0,0.0,1.0,1.0,7.0,0.0,1.0,0.0,0.0,4.0,4.0,3.0,7.0,3.0,3.0,7.0,1.0,1.0,3.0,3.0,1.0,1.0,3.0,0.0,9.0,2.0,7.0,2.0,2.0,2.0,2.0,0.0,1.0,2.0,3.0,2.0,2.0


In [17]:
np.fill_diagonal(cowatch_df2.values, 0)
cowatch_df2.head()


##movie-movie matrix 완성!

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,...,1642,1643,1644,1645,1646,1647,1648,1649,1650,1651,1652,1653,1654,1655,1656,1657,1658,1659,1660,1661,1662,1663,1664,1665,1666,1667,1668,1669,1670,1671,1672,1673,1674,1675,1676,1677,1678,1679,1680,1681
0,0.0,61.0,325.0,8.0,14.0,6.0,7.0,4.0,10.0,12.0,25.0,26.0,43.0,67.0,30.0,72.0,83.0,27.0,76.0,10.0,105.0,33.0,20.0,20.0,45.0,20.0,20.0,13.0,20.0,2.0,17.0,2.0,1.0,116.0,10.0,14.0,16.0,6.0,13.0,30.0,...,30.0,5.0,14.0,22.0,10.0,14.0,58.0,179.0,5.0,18.0,23.0,2.0,26.0,36.0,10.0,36.0,21.0,27.0,244.0,17.0,7.0,12.0,11.0,23.0,17.0,15.0,2.0,47.0,17.0,134.0,13.0,11.0,3.0,58.0,4.0,12.0,8.0,12.0,14.0,7.0
1,61.0,0.0,74.0,1.0,3.0,2.0,2.0,0.0,9.0,7.0,14.0,12.0,22.0,15.0,15.0,21.0,13.0,3.0,16.0,4.0,15.0,12.0,10.0,8.0,7.0,13.0,16.0,4.0,3.0,3.0,6.0,1.0,1.0,19.0,0.0,5.0,4.0,0.0,1.0,5.0,...,9.0,4.0,6.0,7.0,2.0,3.0,12.0,38.0,5.0,16.0,10.0,0.0,4.0,6.0,2.0,10.0,8.0,9.0,48.0,7.0,4.0,4.0,6.0,10.0,10.0,2.0,1.0,9.0,4.0,25.0,7.0,4.0,1.0,4.0,1.0,6.0,1.0,2.0,5.0,0.0
2,325.0,74.0,0.0,7.0,16.0,6.0,5.0,6.0,17.0,20.0,37.0,36.0,53.0,55.0,43.0,86.0,86.0,31.0,76.0,11.0,103.0,50.0,25.0,24.0,32.0,29.0,29.0,16.0,22.0,4.0,17.0,1.0,1.0,113.0,7.0,10.0,12.0,4.0,10.0,23.0,...,36.0,6.0,17.0,22.0,10.0,13.0,49.0,185.0,5.0,24.0,23.0,3.0,24.0,34.0,10.0,40.0,24.0,31.0,290.0,20.0,6.0,15.0,12.0,24.0,20.0,18.0,2.0,46.0,15.0,125.0,14.0,14.0,3.0,39.0,5.0,14.0,6.0,11.0,12.0,5.0
3,8.0,1.0,7.0,0.0,4.0,2.0,1.0,0.0,0.0,1.0,1.0,0.0,0.0,4.0,1.0,4.0,2.0,2.0,4.0,0.0,6.0,2.0,1.0,1.0,2.0,0.0,0.0,0.0,3.0,0.0,1.0,0.0,1.0,6.0,1.0,0.0,4.0,1.0,3.0,1.0,...,1.0,1.0,0.0,1.0,0.0,0.0,3.0,8.0,0.0,1.0,0.0,0.0,2.0,2.0,2.0,3.0,1.0,1.0,5.0,0.0,0.0,2.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,7.0,1.0,0.0,2.0,3.0,0.0,0.0,5.0,3.0,3.0,4.0
4,14.0,3.0,16.0,4.0,0.0,4.0,1.0,0.0,0.0,0.0,2.0,1.0,2.0,1.0,2.0,6.0,6.0,2.0,4.0,1.0,5.0,6.0,1.0,2.0,0.0,0.0,0.0,2.0,3.0,0.0,4.0,1.0,1.0,11.0,0.0,3.0,2.0,0.0,4.0,6.0,...,1.0,1.0,0.0,0.0,0.0,1.0,1.0,7.0,0.0,1.0,0.0,0.0,4.0,4.0,3.0,7.0,3.0,3.0,7.0,1.0,1.0,3.0,3.0,1.0,1.0,3.0,0.0,9.0,2.0,7.0,2.0,2.0,2.0,2.0,0.0,1.0,2.0,3.0,2.0,2.0


In [30]:
from sklearn.metrics.pairwise import linear_kernel
# movie_movie_ratings_df = linear_kernel(user_movie_ratings_df, user_movie_ratings_df)
cowatch = linear_kernel(movie_user_watch_df, movie_user_watch_df)
cowatch_df = pd.DataFrame(cowatch,index = movie_user_ratings_df.index, columns = movie_user_ratings_df.index)
np.fill_diagonal(cowatch_df.values, 0)
cowatch_df.head()

movie_id,1,10,100,1000,1001,1002,1003,1004,1005,1006,1007,1008,1009,101,1010,1011,1012,1013,1014,1015,1016,1017,1018,1019,102,1020,1021,1022,1023,1024,1025,1026,1027,1028,1029,103,1030,1031,1032,1033,...,963,964,965,966,967,968,969,97,970,971,972,973,974,975,976,977,978,979,98,980,981,982,983,984,985,986,987,988,989,99,990,991,992,993,994,995,996,997,998,999
movie_id,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
1,0.0,61.0,325.0,8.0,14.0,6.0,7.0,4.0,10.0,12.0,25.0,26.0,43.0,67.0,30.0,72.0,83.0,27.0,76.0,10.0,105.0,33.0,20.0,20.0,45.0,20.0,20.0,13.0,20.0,2.0,17.0,2.0,1.0,116.0,10.0,14.0,16.0,6.0,13.0,30.0,...,30.0,5.0,14.0,22.0,10.0,14.0,58.0,179.0,5.0,18.0,23.0,2.0,26.0,36.0,10.0,36.0,21.0,27.0,244.0,17.0,7.0,12.0,11.0,23.0,17.0,15.0,2.0,47.0,17.0,134.0,13.0,11.0,3.0,58.0,4.0,12.0,8.0,12.0,14.0,7.0
10,61.0,0.0,74.0,1.0,3.0,2.0,2.0,0.0,9.0,7.0,14.0,12.0,22.0,15.0,15.0,21.0,13.0,3.0,16.0,4.0,15.0,12.0,10.0,8.0,7.0,13.0,16.0,4.0,3.0,3.0,6.0,1.0,1.0,19.0,0.0,5.0,4.0,0.0,1.0,5.0,...,9.0,4.0,6.0,7.0,2.0,3.0,12.0,38.0,5.0,16.0,10.0,0.0,4.0,6.0,2.0,10.0,8.0,9.0,48.0,7.0,4.0,4.0,6.0,10.0,10.0,2.0,1.0,9.0,4.0,25.0,7.0,4.0,1.0,4.0,1.0,6.0,1.0,2.0,5.0,0.0
100,325.0,74.0,0.0,7.0,16.0,6.0,5.0,6.0,17.0,20.0,37.0,36.0,53.0,55.0,43.0,86.0,86.0,31.0,76.0,11.0,103.0,50.0,25.0,24.0,32.0,29.0,29.0,16.0,22.0,4.0,17.0,1.0,1.0,113.0,7.0,10.0,12.0,4.0,10.0,23.0,...,36.0,6.0,17.0,22.0,10.0,13.0,49.0,185.0,5.0,24.0,23.0,3.0,24.0,34.0,10.0,40.0,24.0,31.0,290.0,20.0,6.0,15.0,12.0,24.0,20.0,18.0,2.0,46.0,15.0,125.0,14.0,14.0,3.0,39.0,5.0,14.0,6.0,11.0,12.0,5.0
1000,8.0,1.0,7.0,0.0,4.0,2.0,1.0,0.0,0.0,1.0,1.0,0.0,0.0,4.0,1.0,4.0,2.0,2.0,4.0,0.0,6.0,2.0,1.0,1.0,2.0,0.0,0.0,0.0,3.0,0.0,1.0,0.0,1.0,6.0,1.0,0.0,4.0,1.0,3.0,1.0,...,1.0,1.0,0.0,1.0,0.0,0.0,3.0,8.0,0.0,1.0,0.0,0.0,2.0,2.0,2.0,3.0,1.0,1.0,5.0,0.0,0.0,2.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,7.0,1.0,0.0,2.0,3.0,0.0,0.0,5.0,3.0,3.0,4.0
1001,14.0,3.0,16.0,4.0,0.0,4.0,1.0,0.0,0.0,0.0,2.0,1.0,2.0,1.0,2.0,6.0,6.0,2.0,4.0,1.0,5.0,6.0,1.0,2.0,0.0,0.0,0.0,2.0,3.0,0.0,4.0,1.0,1.0,11.0,0.0,3.0,2.0,0.0,4.0,6.0,...,1.0,1.0,0.0,0.0,0.0,1.0,1.0,7.0,0.0,1.0,0.0,0.0,4.0,4.0,3.0,7.0,3.0,3.0,7.0,1.0,1.0,3.0,3.0,1.0,1.0,3.0,0.0,9.0,2.0,7.0,2.0,2.0,2.0,2.0,0.0,1.0,2.0,3.0,2.0,2.0


In [19]:
cowatch_df.shape

(1682, 1682)

# movie_id movie_title movie_vector 만들기

In [20]:
movie_title_vector_df = ratings_df[['movie_title']]
movie_title_vector_df.head()

,movie_title
0,One Flew Over the Cuckoo's Nest (1975)
1,Strictly Ballroom (1992)
2,"Very Brady Sequel, A (1996)"
3,Pulp Fiction (1994)
4,Scream 2 (1997)


In [21]:
movie_title_vector_df['movie_vector'] = movie_title_vector_df['movie_title'].apply(lambda x: np.array(embed([x]))[0])
movie_title_vector_df.head()

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


,movie_title,movie_vector
0,One Flew Over the Cuckoo's Nest (1975),"[-0.041987445, -0.16832094, -0.0048619434, -0...."
1,Strictly Ballroom (1992),"[0.1138464, 0.02684559, -0.0785413, 0.02889860..."
2,"Very Brady Sequel, A (1996)","[-0.03591001, -0.022352844, -0.015582499, -0.0..."
3,Pulp Fiction (1994),"[-0.014452995, -0.1380628, -0.08766341, 0.0422..."
4,Scream 2 (1997),"[-0.019198399, 0.0065805796, -0.104583174, 0.0..."


In [ ]:
# print(movie_title_vector_df.loc[movie_title_vector_df['movie_title']== 'You So Crazy (1994)', 'movie_vector'])

17933    [-0.124294996, 0.055389643, 0.024181958, -0.03...
Name: movie_vector, dtype: object


In [22]:
movie_title_vector_df['movie_id'] = ratings_df[['movie_id']]

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [23]:
movie_title_vector_df.head()

,movie_title,movie_vector,movie_id
0,One Flew Over the Cuckoo's Nest (1975),"[-0.041987445, -0.16832094, -0.0048619434, -0....",357
1,Strictly Ballroom (1992),"[0.1138464, 0.02684559, -0.0785413, 0.02889860...",709
2,"Very Brady Sequel, A (1996)","[-0.03591001, -0.022352844, -0.015582499, -0.0...",412
3,Pulp Fiction (1994),"[-0.014452995, -0.1380628, -0.08766341, 0.0422...",56
4,Scream 2 (1997),"[-0.019198399, 0.0065805796, -0.104583174, 0.0...",895


# split dataset into train/vali/eval

In [29]:
cowatch_df.head()

cowatch
seed candidate         
1    1              0.0
     10            61.0
     100          325.0
     1000           8.0
     1001          14.0

In [ ]:
train_ratio = 0.8
val_ratio = 0.1
test_ratio = 0.1

train_index = cowatch_df.shape[0] * train_ratio
val_index = train_index + cowatch_df.shape[0] * val_ratio
test_index = val_index + cowatch_df.shape[0] * test_ratio

train_cowatch_df = cowatch_df[:train_index][:train_index]


In [37]:
temp = cowatch_df[:2][:3]
temp

movie_id,1,10,100,1000,1001,1002,1003,1004,1005,1006,1007,1008,1009,101,1010,1011,1012,1013,1014,1015,1016,1017,1018,1019,102,1020,1021,1022,1023,1024,1025,1026,1027,1028,1029,103,1030,1031,1032,1033,...,963,964,965,966,967,968,969,97,970,971,972,973,974,975,976,977,978,979,98,980,981,982,983,984,985,986,987,988,989,99,990,991,992,993,994,995,996,997,998,999
movie_id,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
1,0.0,61.0,325.0,8.0,14.0,6.0,7.0,4.0,10.0,12.0,25.0,26.0,43.0,67.0,30.0,72.0,83.0,27.0,76.0,10.0,105.0,33.0,20.0,20.0,45.0,20.0,20.0,13.0,20.0,2.0,17.0,2.0,1.0,116.0,10.0,14.0,16.0,6.0,13.0,30.0,...,30.0,5.0,14.0,22.0,10.0,14.0,58.0,179.0,5.0,18.0,23.0,2.0,26.0,36.0,10.0,36.0,21.0,27.0,244.0,17.0,7.0,12.0,11.0,23.0,17.0,15.0,2.0,47.0,17.0,134.0,13.0,11.0,3.0,58.0,4.0,12.0,8.0,12.0,14.0,7.0
10,61.0,0.0,74.0,1.0,3.0,2.0,2.0,0.0,9.0,7.0,14.0,12.0,22.0,15.0,15.0,21.0,13.0,3.0,16.0,4.0,15.0,12.0,10.0,8.0,7.0,13.0,16.0,4.0,3.0,3.0,6.0,1.0,1.0,19.0,0.0,5.0,4.0,0.0,1.0,5.0,...,9.0,4.0,6.0,7.0,2.0,3.0,12.0,38.0,5.0,16.0,10.0,0.0,4.0,6.0,2.0,10.0,8.0,9.0,48.0,7.0,4.0,4.0,6.0,10.0,10.0,2.0,1.0,9.0,4.0,25.0,7.0,4.0,1.0,4.0,1.0,6.0,1.0,2.0,5.0,0.0


# seed , candidate, cowatch dataframe 만들기

In [24]:
cowatch_df.head()

movie_id,1,10,100,1000,1001,1002,1003,1004,1005,1006,1007,1008,1009,101,1010,1011,1012,1013,1014,1015,1016,1017,1018,1019,102,1020,1021,1022,1023,1024,1025,1026,1027,1028,1029,103,1030,1031,1032,1033,...,963,964,965,966,967,968,969,97,970,971,972,973,974,975,976,977,978,979,98,980,981,982,983,984,985,986,987,988,989,99,990,991,992,993,994,995,996,997,998,999
movie_id,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
1,0.0,61.0,325.0,8.0,14.0,6.0,7.0,4.0,10.0,12.0,25.0,26.0,43.0,67.0,30.0,72.0,83.0,27.0,76.0,10.0,105.0,33.0,20.0,20.0,45.0,20.0,20.0,13.0,20.0,2.0,17.0,2.0,1.0,116.0,10.0,14.0,16.0,6.0,13.0,30.0,...,30.0,5.0,14.0,22.0,10.0,14.0,58.0,179.0,5.0,18.0,23.0,2.0,26.0,36.0,10.0,36.0,21.0,27.0,244.0,17.0,7.0,12.0,11.0,23.0,17.0,15.0,2.0,47.0,17.0,134.0,13.0,11.0,3.0,58.0,4.0,12.0,8.0,12.0,14.0,7.0
10,61.0,0.0,74.0,1.0,3.0,2.0,2.0,0.0,9.0,7.0,14.0,12.0,22.0,15.0,15.0,21.0,13.0,3.0,16.0,4.0,15.0,12.0,10.0,8.0,7.0,13.0,16.0,4.0,3.0,3.0,6.0,1.0,1.0,19.0,0.0,5.0,4.0,0.0,1.0,5.0,...,9.0,4.0,6.0,7.0,2.0,3.0,12.0,38.0,5.0,16.0,10.0,0.0,4.0,6.0,2.0,10.0,8.0,9.0,48.0,7.0,4.0,4.0,6.0,10.0,10.0,2.0,1.0,9.0,4.0,25.0,7.0,4.0,1.0,4.0,1.0,6.0,1.0,2.0,5.0,0.0
100,325.0,74.0,0.0,7.0,16.0,6.0,5.0,6.0,17.0,20.0,37.0,36.0,53.0,55.0,43.0,86.0,86.0,31.0,76.0,11.0,103.0,50.0,25.0,24.0,32.0,29.0,29.0,16.0,22.0,4.0,17.0,1.0,1.0,113.0,7.0,10.0,12.0,4.0,10.0,23.0,...,36.0,6.0,17.0,22.0,10.0,13.0,49.0,185.0,5.0,24.0,23.0,3.0,24.0,34.0,10.0,40.0,24.0,31.0,290.0,20.0,6.0,15.0,12.0,24.0,20.0,18.0,2.0,46.0,15.0,125.0,14.0,14.0,3.0,39.0,5.0,14.0,6.0,11.0,12.0,5.0
1000,8.0,1.0,7.0,0.0,4.0,2.0,1.0,0.0,0.0,1.0,1.0,0.0,0.0,4.0,1.0,4.0,2.0,2.0,4.0,0.0,6.0,2.0,1.0,1.0,2.0,0.0,0.0,0.0,3.0,0.0,1.0,0.0,1.0,6.0,1.0,0.0,4.0,1.0,3.0,1.0,...,1.0,1.0,0.0,1.0,0.0,0.0,3.0,8.0,0.0,1.0,0.0,0.0,2.0,2.0,2.0,3.0,1.0,1.0,5.0,0.0,0.0,2.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,7.0,1.0,0.0,2.0,3.0,0.0,0.0,5.0,3.0,3.0,4.0
1001,14.0,3.0,16.0,4.0,0.0,4.0,1.0,0.0,0.0,0.0,2.0,1.0,2.0,1.0,2.0,6.0,6.0,2.0,4.0,1.0,5.0,6.0,1.0,2.0,0.0,0.0,0.0,2.0,3.0,0.0,4.0,1.0,1.0,11.0,0.0,3.0,2.0,0.0,4.0,6.0,...,1.0,1.0,0.0,0.0,0.0,1.0,1.0,7.0,0.0,1.0,0.0,0.0,4.0,4.0,3.0,7.0,3.0,3.0,7.0,1.0,1.0,3.0,3.0,1.0,1.0,3.0,0.0,9.0,2.0,7.0,2.0,2.0,2.0,2.0,0.0,1.0,2.0,3.0,2.0,2.0


In [25]:
cowatch_df.shape

(1682, 1682)

In [26]:
cowatch_df = pd.DataFrame(cowatch_df.unstack())
cowatch_df.columns = ['cowatch']
cowatch_df.index.names = ['seed','candidate']
cowatch_df.head()

cowatch
seed candidate         
1    1              0.0
     10            61.0
     100          325.0
     1000           8.0
     1001          14.0

In [ ]:
cowatch_df2 = cowatch_df.reset_index()
cowatch_df2.head(10)

,seed,candidate,cowatch
0,1,1,0.0
1,1,10,61.0
2,1,100,325.0
3,1,1000,8.0
4,1,1001,14.0
5,1,1002,6.0
6,1,1003,7.0
7,1,1004,4.0
8,1,1005,10.0
9,1,1006,12.0


In [ ]:
cowatch_df2.shape

(2829124, 3)

In [ ]:
cowatch_df2['cowatch'].describe()

count    2.829124e+06
mean     7.104960e+00
std      1.638444e+01
min      0.000000e+00
25%      0.000000e+00
50%      1.000000e+00
75%      6.000000e+00
max      4.800000e+02
Name: cowatch, dtype: float64

In [ ]:
cowatch_df3 = cowatch_df2.apply(lambda x: x)

seed
1       [1, 10, 100, 1000, 1001, 1002, 1003, 1004, 100...
10      [1, 10, 100, 1000, 1001, 1002, 1003, 1004, 100...
100     [1, 10, 100, 1000, 1001, 1002, 1003, 1004, 100...
1000    [1, 10, 100, 1000, 1001, 1002, 1003, 1004, 100...
1001    [1, 10, 100, 1000, 1001, 1002, 1003, 1004, 100...
                              ...                        
995     [1, 10, 100, 1000, 1001, 1002, 1003, 1004, 100...
996     [1, 10, 100, 1000, 1001, 1002, 1003, 1004, 100...
997     [1, 10, 100, 1000, 1001, 1002, 1003, 1004, 100...
998     [1, 10, 100, 1000, 1001, 1002, 1003, 1004, 100...
999     [1, 10, 100, 1000, 1001, 1002, 1003, 1004, 100...
Name: candidate, Length: 1682, dtype: object

In [ ]:
def label_func(x):
  if x > 11:
    return 3
  elif x > 7:
    return 2
  elif x > 0:
    return 1
  else:
    return 0